In [ ]:
library(RAnEn)
library(maps)

stopifnot(packageVersion('RAnEn')>="3.5.0")


## Introduction

This article walks you through the basic usage of the `RAnEn` library. This exercise uses short-term surface temperature forecasts as an example. Recommend using [binder](https://mybinder.org/badge.svg) and the corresponding `.ipynb` file will guide you through the script line by line.

You will learn how to use these functions:

- `generateConfiguration`
- `generateAnalogs`
- `verify*` functions

## Data Description

First, let's load the prepared NAM model forecasts and analysis. You can find the data in this direct folder if you want to run this script locally.



In [ ]:
file.name <- 'data-NAM-StateCollege.RData'
# file.name <- 'data-NAM-Denver.RData'
# file.name <- 'data-METAR-CONUS.Rdata'

if (!file.exists(file.name)) {
  cat('Downloading from the data server which might be slow ...\n')
  download.file(url = paste('https://prosecco.geog.psu.edu/', file.name, sep = ''),
                destfile = file.name)
}
load(file.name)
rm(file.name)

print(ls())


The following parameters are included and provided in the dataset.



In [ ]:
print(forecasts$ParameterNames)


Foreacsts and analysis have the following time span. Because the original time values record the number of seconds since an original time, the function `as.POSIXct` is used to convert them to readable date-time class objects. Alternatively, `RAnEn::toDateTime` can complete the same action. It is a convenient wrapper function of `as.POSIXct`.



In [ ]:
print(as.POSIXct(range(forecasts$Times), origin = '1970-01-01', tz = 'UTC'))
print(as.POSIXct(range(observations$Times), origin = '1970-01-01', tz = 'UTC'))


Coordinates in lat/lon are provided for each grid point. We can plot them on maps. **Note that coordinates start from the bottom left and proceed in a row-wise order**. There are 11 rows and 11 columns of grid points in the data sets.



In [ ]:
map('county', regions = 'pennsylvania', col = 'grey')
points(forecasts$Xs - 360, forecasts$Ys, pch = 16, cex = .3, col = 'red')


Finally, let's look at the dimensions of forecasts and analysis.



In [ ]:
print(dim(forecasts$Data))
print(dim(observations$Data))


Forecasts have 4 dimensions which include parameters, stations/grid points, days, and Forecast Lead Times (FLTs). Analysis have 3 dimensions which include parameters, stations/grid points, and time.

## Generate Temperature Forecasts

Now that we have a general idea of the data, we can generate temperature forecasts. The steps to generate AnEn forecasts are as follow:

- Initialize a configuration
- Set up options
- Generate AnEn

First, we set up the configuration. We generate an AnEn for one day using about one year of historical data for all the grid points available. For a full list of avaiable options, please check the document for [RAnEn::generateConfiguration](https://weiming-hu.github.io/AnalogsEnsemble/R/reference/generateConfiguration.html).



In [ ]:
# We use independent search configuration.
config <- generateConfiguration('independentSearch')

# Set up the start and end indices for test times
test.start <- 366
test.end <- 372

# Set up the start and end indices for search times
search.start <- 1
search.end <- 365

# Set up forecasts and the time, FLT information
config$forecasts <- forecasts$Data
config$forecast_times <- forecasts$Times
config$flts <- forecasts$FLTs

# Set up observations and the time information
config$search_observations <- observations$Data
config$observation_times <- observations$Times

# Set up the number of members to keep in the analog ensemble
# Empirically, the number of members is the square root of number of search times.
config$num_members <- sqrt(search.end - search.start + 1)

# Set up the variable that we want to generate AnEn for.
# This is the index of parameter in observation parameter names.
#
config$observation_id <- 8

# Set up weights for each parameters
config$weights <- c(1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1)

# Set the test times to generate AnEn for
config$test_times_compare <- forecasts$Times[test.start:test.end]

# Set the search times to generate AnEn from
config$search_times_compare <- forecasts$Times[search.start:search.end]

# Set circular variable if there is any
if ('ParameterCirculars' %in% names(forecasts)) {
  config$circulars <- unlist(lapply(forecasts$ParameterCirculars, function(x) {
    return(which(x == forecasts$ParameterNames))}))
}


Now we can generate an AnEn. This could take a while. On a Mac Air, it takes about 2 minutes.



In [ ]:
AnEn <- generateAnalogs(config)


The result variable `AnEn` is a list of members. Let's check what the members are.



In [ ]:
print(AnEn)


## Verification

Finally, let's extract the AnEn forecast values and the corresponding observations.



In [ ]:
# Collect analog ensemble values
anen <- AnEn$analogs[, , , , 1]

# Collect observations for the test period and the forecasted weather variable
obs <- alignObservations(observations$Data, observations$Times,
                         config$test_times_compare, config$flts,
                         silent = T, show.progress = F)
obs <- obs[config$observation_id, , , ]


Then we can generate verification statistics, and see how AnEn is performing.



In [ ]:
# Generate verification metrics
ret.MAE <- verifyMAE(anen, obs)
ret.RMSE <- verifyRMSE(anen, obs)
ret.Bias <- verifyBias(anen, obs)
ret.RH <- verifyRankHist(anen, obs)

# Let's make some figures
par(mfrow = c(4, 1), mar = c(3, 4.5, 1, 1))
plot(config$flts/3600, ret.MAE$flt, type = 'b', pch = 1, cex = 0.5,
     xlab = '', ylab = 'MAE')
plot(config$flts/3600, ret.RMSE$flt, type = 'b', pch = 1, cex = 0.5,
     xlab = '', ylab = 'RMSE')
plot(config$flts/3600, ret.Bias$flt, type = 'b', pch = 1, cex = 0.5,
     xlab = 'Lead Times (h)', ylab = 'Bias')
barplot(ret.RH$rank, ylab = 'Rank Frequency')